In [1]:
import pandas as pd
import datetime
import glob
import sqlite3
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, distinct

In [2]:
all_csv = glob.glob('../Desktop/AQI_project/2020_aqi/*.csv')

print(all_csv)
    

['../Desktop/AQI_project/2020_aqi/ad_viz_plotval_data (2).csv', '../Desktop/AQI_project/2020_aqi/ad_viz_plotval_data (3).csv', '../Desktop/AQI_project/2020_aqi/ad_viz_plotval_data (4).csv', '../Desktop/AQI_project/2020_aqi/ad_viz_plotval_data (5).csv', '../Desktop/AQI_project/2020_aqi/ad_viz_plotval_data.csv', '../Desktop/AQI_project/2020_aqi/ad_viz_plotval_data (6).csv', '../Desktop/AQI_project/2020_aqi/ad_viz_plotval_data (1).csv']


In [3]:
appended_data = []

for i in all_csv:
    
    csv_path = i
    csv = pd.read_csv(csv_path)
    appended_data.append(csv)
    
apd = pd.concat(appended_data)



In [4]:
air_2020 = pd.DataFrame(apd)

air_2020.head()

,Date,Source,Site ID,POC,Daily Mean Pb Concentration,UNITS,DAILY_AQI_VALUE,Site Name,DAILY_OBS_COUNT,PERCENT_COMPLETE,...,COUNTY_CODE,COUNTY,SITE_LATITUDE,SITE_LONGITUDE,Daily Max 1-hour NO2 Concentration,Daily Mean PM10 Concentration,Daily Mean PM2.5 Concentration,Daily Max 8-hour Ozone Concentration,Daily Max 1-hour SO2 Concentration,Daily Max 8-hour CO Concentration
0,01/04/2020,AQS,390290019,1,0.002,ug/m3 LC,.,Port Authority,1,100.0,...,29,Columbiana,40.631545,-80.547181,NaN,NaN,NaN,NaN,NaN,NaN
1,01/10/2020,AQS,390290019,1,0.004,ug/m3 LC,.,Port Authority,1,100.0,...,29,Columbiana,40.631545,-80.547181,NaN,NaN,NaN,NaN,NaN,NaN
2,01/16/2020,AQS,390290019,1,0.002,ug/m3 LC,.,Port Authority,1,100.0,...,29,Columbiana,40.631545,-80.547181,NaN,NaN,NaN,NaN,NaN,NaN
3,01/22/2020,AQS,390290019,1,0.007,ug/m3 LC,.,Port Authority,1,100.0,...,29,Columbiana,40.631545,-80.547181,NaN,NaN,NaN,NaN,NaN,NaN
4,01/28/2020,AQS,390290019,1,0.002,ug/m3 LC,.,Port Authority,1,100.0,...,29,Columbiana,40.631545,-80.547181,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
air_2020.dtypes

Date                                     object
Source                                   object
Site ID                                   int64
POC                                       int64
Daily Mean Pb Concentration             float64
UNITS                                    object
DAILY_AQI_VALUE                          object
Site Name                                object
DAILY_OBS_COUNT                           int64
PERCENT_COMPLETE                        float64
AQS_PARAMETER_CODE                        int64
AQS_PARAMETER_DESC                       object
CBSA_CODE                               float64
CBSA_NAME                                object
STATE_CODE                                int64
STATE                                    object
COUNTY_CODE                               int64
COUNTY                                   object
SITE_LATITUDE                           float64
SITE_LONGITUDE                          float64
Daily Max 1-hour NO2 Concentration      

In [6]:
cols = air_2020.columns.to_list()

cols

['Date',
 'Source',
 'Site ID',
 'POC',
 'Daily Mean Pb Concentration',
 'UNITS',
 'DAILY_AQI_VALUE',
 'Site Name',
 'DAILY_OBS_COUNT',
 'PERCENT_COMPLETE',
 'AQS_PARAMETER_CODE',
 'AQS_PARAMETER_DESC',
 'CBSA_CODE',
 'CBSA_NAME',
 'STATE_CODE',
 'STATE',
 'COUNTY_CODE',
 'COUNTY',
 'SITE_LATITUDE',
 'SITE_LONGITUDE',
 'Daily Max 1-hour NO2 Concentration',
 'Daily Mean PM10 Concentration',
 'Daily Mean PM2.5 Concentration',
 'Daily Max 8-hour Ozone Concentration',
 'Daily Max 1-hour SO2 Concentration',
 'Daily Max 8-hour CO Concentration']

In [7]:
site_info = air_2020.drop(['Date','POC','AQS_PARAMETER_DESC','DAILY_AQI_VALUE','DAILY_OBS_COUNT','AQS_PARAMETER_CODE','PERCENT_COMPLETE','Daily Mean Pb Concentration','Daily Max 1-hour NO2 Concentration', 'UNITS','Daily Mean PM10 Concentration','Daily Mean PM2.5 Concentration','Daily Max 8-hour Ozone Concentration','Daily Max 1-hour SO2 Concentration', 'Daily Max 8-hour CO Concentration'], axis = 1)



In [8]:
site_info = site_info.drop_duplicates()

site_info.head()

,Source,Site ID,Site Name,CBSA_CODE,CBSA_NAME,STATE_CODE,STATE,COUNTY_CODE,COUNTY,SITE_LATITUDE,SITE_LONGITUDE
0,AQS,390290019,Port Authority,41400.0,"Salem, OH",39,Ohio,29,Columbiana,40.631545,-80.547181
5,AQS,390290020,East Liverpool WTP,41400.0,"Salem, OH",39,Ohio,29,Columbiana,40.639509,-80.523951
16,AQS,390290023,Eastside Elem,41400.0,"Salem, OH",39,Ohio,29,Columbiana,40.634688,-80.545695
26,AQS,390350038,St Theodosius,17460.0,"Cleveland-Elyria, OH",39,Ohio,35,Cuyahoga,41.477011,-81.682383
31,AQS,390350049,Ferro,17460.0,"Cleveland-Elyria, OH",39,Ohio,35,Cuyahoga,41.446342,-81.650700


In [9]:
lead_2020 = air_2020.drop(['Site Name','Source','CBSA_NAME','CBSA_CODE','STATE_CODE','STATE', 'COUNTY_CODE', 'COUNTY', 'SITE_LATITUDE','SITE_LONGITUDE','Daily Max 1-hour NO2 Concentration', 'UNITS','Daily Mean PM10 Concentration','Daily Mean PM2.5 Concentration','Daily Max 8-hour Ozone Concentration','Daily Max 1-hour SO2 Concentration', 'Daily Max 8-hour CO Concentration'], axis =1)
                          
                          

In [10]:
no2_2020 = air_2020.drop(['Site Name','Source','CBSA_NAME','CBSA_CODE','STATE_CODE','STATE', 'COUNTY_CODE', 'COUNTY', 'SITE_LATITUDE','SITE_LONGITUDE','Daily Mean Pb Concentration', 'UNITS','Daily Mean PM10 Concentration','Daily Mean PM2.5 Concentration','Daily Max 8-hour Ozone Concentration','Daily Max 1-hour SO2 Concentration', 'Daily Max 8-hour CO Concentration'], axis = 1)


In [11]:
pm10_2020 = air_2020.drop(['Site Name','Source','CBSA_NAME','CBSA_CODE','STATE_CODE','STATE', 'COUNTY_CODE', 'COUNTY', 'SITE_LATITUDE','SITE_LONGITUDE','Daily Mean Pb Concentration','Daily Max 1-hour NO2 Concentration' ,'Daily Mean PM2.5 Concentration','Daily Max 8-hour Ozone Concentration','Daily Max 1-hour SO2 Concentration', 'Daily Max 8-hour CO Concentration'], axis = 1)


In [12]:
pm2_2020 = air_2020.drop(['Site Name','Source','CBSA_NAME','CBSA_CODE','STATE_CODE','STATE', 'COUNTY_CODE', 'COUNTY', 'SITE_LATITUDE','SITE_LONGITUDE','Daily Mean Pb Concentration','Daily Max 1-hour NO2 Concentration' ,'Daily Mean PM10 Concentration','Daily Max 8-hour Ozone Concentration','Daily Max 1-hour SO2 Concentration', 'Daily Max 8-hour CO Concentration'], axis = 1)


In [13]:
ozone_2020 = air_2020.drop(['Site Name','Source','CBSA_NAME','CBSA_CODE','STATE_CODE','STATE', 'COUNTY_CODE', 'COUNTY', 'SITE_LATITUDE','SITE_LONGITUDE','Daily Mean Pb Concentration','Daily Max 1-hour NO2 Concentration' ,'Daily Mean PM2.5 Concentration','Daily Mean PM10 Concentration','Daily Max 1-hour SO2 Concentration', 'Daily Max 8-hour CO Concentration'], axis = 1)


In [14]:
so2_2020 = air_2020.drop(['Site Name','Source','CBSA_NAME','CBSA_CODE','STATE_CODE','STATE', 'COUNTY_CODE', 'COUNTY', 'SITE_LATITUDE','SITE_LONGITUDE','Daily Mean Pb Concentration','Daily Max 1-hour NO2 Concentration' ,'Daily Mean PM2.5 Concentration','Daily Max 8-hour Ozone Concentration','Daily Mean PM10 Concentration', 'Daily Max 8-hour CO Concentration'], axis = 1)


In [15]:
co_2020 = air_2020.drop(['Site Name','Source','CBSA_NAME','CBSA_CODE','STATE_CODE','STATE', 'COUNTY_CODE', 'COUNTY', 'SITE_LATITUDE','SITE_LONGITUDE','Daily Mean Pb Concentration','Daily Max 1-hour NO2 Concentration' ,'Daily Mean PM2.5 Concentration','Daily Max 8-hour Ozone Concentration','Daily Max 1-hour SO2 Concentration', 'Daily Mean PM10 Concentration'], axis = 1)


In [16]:
lead_2020.head()

,Date,Site ID,POC,Daily Mean Pb Concentration,DAILY_AQI_VALUE,DAILY_OBS_COUNT,PERCENT_COMPLETE,AQS_PARAMETER_CODE,AQS_PARAMETER_DESC
0,01/04/2020,390290019,1,0.002,.,1,100.0,14129,Lead (TSP) LC
1,01/10/2020,390290019,1,0.004,.,1,100.0,14129,Lead (TSP) LC
2,01/16/2020,390290019,1,0.002,.,1,100.0,14129,Lead (TSP) LC
3,01/22/2020,390290019,1,0.007,.,1,100.0,14129,Lead (TSP) LC
4,01/28/2020,390290019,1,0.002,.,1,100.0,14129,Lead (TSP) LC


In [17]:
conn = sqlite3.connect('AQI_2020_Data')

c = conn.cursor()

In [18]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

In [19]:
get_df_name(lead_2020)

'lead_2020'

In [20]:
def column_names(df):
    all_cols = []
    col_list = df.columns.to_list()
    for i in col_list:
        clean_item = str(i).replace(" ","_")
        all_cols.append(clean_item)
            
    final_list = str(all_cols).replace("[","(").replace("'","").replace("]",")").replace('"','').replace("-","").replace(".","pnt")
    
    
    return final_list

In [21]:
column_names(lead_2020)

'(Date, Site_ID, POC, Daily_Mean_Pb_Concentration, DAILY_AQI_VALUE, DAILY_OBS_COUNT, PERCENT_COMPLETE, AQS_PARAMETER_CODE, AQS_PARAMETER_DESC)'

In [22]:
def new_table(x):
    y = column_names(x)
    
    q = get_df_name(x)
    
    f = str(f"'CREATE TABLE {q} {y}'")
    
    g = f.replace("'","")
    
    c.execute(g)
    
    conn.commit()
    
    x.to_sql( q , conn, if_exists='replace', index = False)
    

In [23]:
def sql_fetch():

    cursorObj = conn.cursor()

    cursorObj.execute('SELECT name from sqlite_master where type= "table"')

    print(cursorObj.fetchall())

sql_fetch()

[]


In [27]:
def add_to_sql(x):
    
    y = get_df_name(x)
    
    x.to_sql(y, conn, if_exists= 'replace', index = False)

In [36]:
new_table(site_info)
add_to_sql(site_info)

In [28]:
new_table(lead_2020)
add_to_sql(lead_2020)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2663: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [29]:
new_table(no2_2020)
add_to_sql(no2_2020)


In [30]:
new_table(pm10_2020)
add_to_sql(pm10_2020)

In [31]:
new_table(pm2_2020)
add_to_sql(pm2_2020)

In [32]:
new_table(ozone_2020)
add_to_sql(ozone_2020)

In [33]:
new_table(so2_2020)
add_to_sql(so2_2020)

In [34]:
new_table(co_2020)
add_to_sql(co_2020)

In [37]:

sql_fetch()

[('lead_2020',), ('no2_2020',), ('pm10_2020',), ('pm2_2020',), ('ozone_2020',), ('so2_2020',), ('co_2020',), ('site_info',)]
